In [ ]:
import pandas as pd
from parepy_toolbox.common_library import sampling

def sampling_generator(number_of_samples: int, numerical_model: dict, variables_settings: list) -> pd.DataFrame:
    """
    Generates random samples for design variables only (X variables), excluding R, S, G, and I columns.

    :param number_of_samples: Number of samples to generate.
    :param numerical_model: Dictionary with sampling configuration (e.g., method and time steps).
    :param variables_settings: List of dictionaries defining the probabilistic distributions.

    :return: DataFrame with generated samples.
    """
    
    # Assegura que todas variáveis tenham uma seed
    for var in variables_settings:
        if 'seed' not in var:
            var['seed'] = None

    n_dimensions = len(variables_settings)
    algorithm = numerical_model['model sampling']
    is_time_analysis = algorithm.upper() in ['MCS-TIME', 'MCS_TIME', 'MCS TIME', 'LHS-TIME', 'LHS_TIME', 'LHS TIME']
    time_analysis = numerical_model.get('time steps', 1)

    # Chamada da função sampling
    results = sampling(
        n_samples=number_of_samples,
        model=numerical_model,
        variables_setup=variables_settings
    )

    # Processa resultados
    if is_time_analysis:
        # Remove a última coluna (STEP)
        results = results[:, :-1]

        # Separa e reestrutura por blocos de tempo
        block_size = time_analysis
        all_rows = []
        for i in range(number_of_samples):
            block = results[i * block_size:(i + 1) * block_size, :].T.flatten().tolist()
            all_rows.append(block)
        results_df = pd.DataFrame(all_rows)

        # Nome das colunas X_{i}_t={t}
        column_names = []
        for i in range(n_dimensions):
            for t in range(time_analysis):
                column_names.append(f'X_{i}_t={t}')

    else:
        results_df = pd.DataFrame(results)
        column_names = [f'X_{i}' for i in range(n_dimensions)]

    results_df.columns = column_names
    return results_df


In [ ]:
numerical_model = {
    'model sampling': 'MCS'
}

variables_settings = [
    {
        'type': 'normal',
        'seed': 42,
        'parameters': {
            'mean': 10,
            'sigma': 2
        }
    },
    {
        'type': 'uniform',
        'seed': 123,
        'parameters': {
            'min': 5,
            'max': 15
        }
    }
]

number_of_samples = 5
df_samples = sampling_generator(number_of_samples, numerical_model, variables_settings)
print(df_samples)

         X_0        X_1
0   9.175257  11.823519
1  12.470282   5.538210
2  11.095637   7.203599
3  10.902782   6.843718
4   6.288076   6.759059
